In [17]:
import pandas as pd

In [18]:
herectvo = pd.read_json("herectvo_raw.json")

In [19]:
herectvo.sample(20)

,Jméno,Bio,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí,Titul,Film,Rok,Úloha,Role,Alternativní jména
34437,Lucia Gažiová (19838),LUCIA KLEIN SVOBODA (vl. jm. Lucia Gažiová; na...,02.02.1979,"Levoča, Československo",None,None,Cinka Panna,Cinka Panna (90196),2008,Hrají,Cinková,"Lucia Svobodová (provdaná), Lucia Klein Svoboda"
63245,Jiří Dvořák (5379),None,04.02.1967,"Brno, Československo",None,None,Princezna zakletá v čase,Princezna zakletá v čase (402091),2020,Mluví,hlas Oživlých skal,None
33858,Miloslav Homola (18975),None,04.02.1923,"Olomouc, Československo",21.09.2006,"Praha, Česká republika",Osvobození Prahy,Osvobození Prahy (397108),1976,"[Mluví, Hrají]","[hlas na Václavském náměstí, občan v tramvaji]",None
71584,Josef Koníček (75651),"JOSEF KONÍČEK (nar. 3.10.1931, Praha – zemřel ...",03.10.1931,"Praha, Československo",03.04.2010,"Praha, Česká republika",Starci na chmelu,Starci na chmelu (396586),1964,"[Choreografie, Zpívá, Hrají]",člen chóru,None
28830,Daniela Šinkorová (14434),None,28.12.1972,"Nitra, Slovensko",None,None,Experti,Experti (37300),2006,Hrají,Helena,None
7223,Emanuel Kovařík (126326),None,13.05.1907,"Horažďovice u Strakonic, Rakousko-Uhersko",04.01.1973,"Praha, Československo",Přednosta stanice,Přednosta stanice (395949),1941,Hrají,posunovač Franta,Emanuel František Kovářík (rodné jméno)
72379,Zdeněk Jarolímek (76404),None,07.08.1926,"Postupice u Benešova, Československo",20.05.2010,"České Budějovice, Česká republika",Svatby pana Voka,Svatby pana Voka (396862),1970,Hrají,patron,None
86212,Hana Čížková (9732),None,13.09.1950,"České Budějovice, Československo",None,None,Kameňák 4,Kameňák 4 (400591),2013,Hrají,Sára Kohnová,None
12377,Michał Pawlicki (126868),None,29.09.1932,"Varšava, Polsko",29.09.2000,"Varšava, Polsko",Kainovo znamení,Kainovo znamení (397639),1989,Hrají,hrabě Rożyński,None
36949,Dana Morávková (2293),None,29.07.1971,"Písek, Československo",None,None,Z pekla štěstí,Z pekla štěstí (53139),1999,"[Choreografie, Hrají]",Dora,None


In [20]:
def rozdel_misto(x):
    if type(x) == str:
        try:
            mesto = x.split(",")[0]
            stat = x.split(",")[1]
        except:
            mesto = None
            stat = x
    else:
        mesto = None
        stat = None
    return [mesto, stat]

In [21]:
herectvo["Město narození"] = herectvo["Místo narození"].apply(lambda x: rozdel_misto(x)[0])
herectvo["Stát narození"] = herectvo["Místo narození"].apply(lambda x: rozdel_misto(x)[1])
herectvo["Město úmrtí"] = herectvo["Místo úmrtí"].apply(lambda x: rozdel_misto(x)[1])
herectvo["Stát úmrtí"] = herectvo["Místo úmrtí"].apply(lambda x: rozdel_misto(x)[1])

In [22]:
herectvo.loc[herectvo["Jméno"] == "Eva Pecková (34563)", "Datum narození"] = None

In [23]:
herectvo["Rok narození"] = herectvo["Datum narození"].astype(str).apply(lambda x: x[-4:])
herectvo["Rok narození"] = pd.to_numeric(herectvo["Rok narození"], errors='coerce') 

In [24]:
herectvo["Datum narození"] = pd.to_datetime(herectvo["Datum narození"], errors="ignore", format = "%d.%m.%Y")

In [25]:
herectvo[herectvo["Jméno"] == "Eva Pecková (34563)"]

,Jméno,Bio,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí,Titul,Film,Rok,Úloha,Role,Alternativní jména,Město narození,Stát narození,Město úmrtí,Stát úmrtí,Rok narození
49343,Eva Pecková (34563),None,None,"Volyně, Československo",18.07.2018,"České Budějovice, Česká republika",Křik,Křik (396542),1963,Hrají,medička,Eva Kubičková (rodné jméno),Volyně,Československo,Česká republika,Česká republika,NaN


In [26]:
herectvo["Datum úmrtí"] = pd.to_datetime(herectvo["Datum úmrtí"], errors="ignore", format = "%d.%m.%Y")

In [27]:
herectvo["Věk"] = herectvo["Rok"].astype(float) - herectvo["Rok narození"].astype(float)

In [28]:
herectvo.to_json("herectvo.json", orient="records")